In [26]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import traceback
import PyPDF2

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [27]:
load_dotenv()

True

In [28]:
key = os.getenv("OPENAI_API_KEY")

In [29]:
llm = ChatOpenAI(
    openai_api_key=key,
    model_name="gpt-3.5-turbo", 
    temperature=0.5)

In [30]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [57]:
TEMPLATE = """

Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to create {number} MCQs
{response_json}

"""

In [58]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
    )

In [59]:
quiz_chain = LLMChain(
    llm=llm, 
    prompt=quiz_generation_prompt, 
    output_key="quiz", 
    verbose=True)

In [60]:
TEMPLATE2 = """

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which need to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check as an expert English Writer of the above quiz:
"""

In [61]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = TEMPLATE2)

In [62]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [63]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"], 
    verbose=True,)

In [64]:
file_path = "/Users/mithunsarkar/Desktop/MITHUN/STUDY/genai/Ineuron/mcqgen/data.txt"

with open(file_path, 'r') as file:
    TEXT = file.read()

In [65]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [87]:
NUMBER = 7 
SUBJECT = "global politics"
TONE = "simple"

In [88]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Text:How should we study politics? Traditionally, there was a tendency to focus on political
actors and institutions at the local and national levels. Beyond this, students and
scholars in the political sub-field of international relations (IR) tend to consider ‘the
international’ as the political space in which these local and national political interests
are represented in the form of interaction between states, regions of states, and a
worldwide ‘states-system’. But since the late twentieth century, the concept of
globalization has challenged these narrow, state-centric ways of thinking about politics.
This book is about global politics, which is to say it is about how politics – struggles
over power, how it should be distributed, and how we might best organise ourselves
and live together as societies – works at the global level.
But what is ‘the global’ when it comes to politics,

In [89]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}") # input token
print(f"Completion Tokens:{cb.completion_tokens}") # output token
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2413
Prompt Tokens:1901
Completion Tokens:512
Total Cost:0.0038755


In [90]:
response

{'text': 'How should we study politics? Traditionally, there was a tendency to focus on political\nactors and institutions at the local and national levels. Beyond this, students and\nscholars in the political sub-field of international relations (IR) tend to consider ‘the\ninternational’ as the political space in which these local and national political interests\nare represented in the form of interaction between states, regions of states, and a\nworldwide ‘states-system’. But since the late twentieth century, the concept of\nglobalization has challenged these narrow, state-centric ways of thinking about politics.\nThis book is about global politics, which is to say it is about how politics – struggles\nover power, how it should be distributed, and how we might best organise ourselves\nand live together as societies – works at the global level.\nBut what is ‘the global’ when it comes to politics, and why does it matter? How does it\ndiffer from ‘the international’, as a way of seeing

In [98]:
quiz_review = response.get("review")
print(quiz_review)

The complexity of the quiz is suitable for global politics students. The questions cover key concepts in the field and require a good understanding of the subject matter. The tone is appropriate for the target audience. No changes are needed.


In [91]:
quiz_str = response.get("quiz")
quiz_str

'{\n"1": {\n"mcq": "What is the main focus of traditional political studies?",\n"options": {\n"a": "Global level politics",\n"b": "Local and national level politics",\n"c": "Regional level politics",\n"d": "State-centric politics"\n},\n"correct": "b"\n},\n"2": {\n"mcq": "What concept has challenged state-centric ways of thinking about politics since the late twentieth century?",\n"options": {\n"a": "Globalization",\n"b": "International relations",\n"c": "Nationalism",\n"d": "Sovereignty"\n},\n"correct": "a"\n},\n"3": {\n"mcq": "What is the main focus of global politics according to the text?",\n"options": {\n"a": "Local politics",\n"b": "National politics",\n"c": "Global level politics",\n"d": "State-centric politics"\n},\n"correct": "c"\n},\n"4": {\n"mcq": "What does the term \'global\' mean in the context of global politics?",\n"options": {\n"a": "Between nations",\n"b": "Worldwide",\n"c": "Comprehensive",\n"d": "Regional"\n},\n"correct": "b"\n},\n"5": {\n"mcq": "What significant cha

In [92]:
quiz_json = json.loads(quiz_str)
quiz_json

{'1': {'mcq': 'What is the main focus of traditional political studies?',
  'options': {'a': 'Global level politics',
   'b': 'Local and national level politics',
   'c': 'Regional level politics',
   'd': 'State-centric politics'},
  'correct': 'b'},
 '2': {'mcq': 'What concept has challenged state-centric ways of thinking about politics since the late twentieth century?',
  'options': {'a': 'Globalization',
   'b': 'International relations',
   'c': 'Nationalism',
   'd': 'Sovereignty'},
  'correct': 'a'},
 '3': {'mcq': 'What is the main focus of global politics according to the text?',
  'options': {'a': 'Local politics',
   'b': 'National politics',
   'c': 'Global level politics',
   'd': 'State-centric politics'},
  'correct': 'c'},
 '4': {'mcq': "What does the term 'global' mean in the context of global politics?",
  'options': {'a': 'Between nations',
   'b': 'Worldwide',
   'c': 'Comprehensive',
   'd': 'Regional'},
  'correct': 'b'},
 '5': {'mcq': 'What significant changes ha

In [93]:
quiz_table = []
for key, value in quiz_json.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table.append({"MCQ": mcq, "Choices": options, "Correct": correct})

quiz_table

[{'MCQ': 'What is the main focus of traditional political studies?',
  'Choices': 'a: Global level politics | b: Local and national level politics | c: Regional level politics | d: State-centric politics',
  'Correct': 'b'},
 {'MCQ': 'What concept has challenged state-centric ways of thinking about politics since the late twentieth century?',
  'Choices': 'a: Globalization | b: International relations | c: Nationalism | d: Sovereignty',
  'Correct': 'a'},
 {'MCQ': 'What is the main focus of global politics according to the text?',
  'Choices': 'a: Local politics | b: National politics | c: Global level politics | d: State-centric politics',
  'Correct': 'c'},
 {'MCQ': "What does the term 'global' mean in the context of global politics?",
  'Choices': 'a: Between nations | b: Worldwide | c: Comprehensive | d: Regional',
  'Correct': 'b'},
 {'MCQ': 'What significant changes have occurred in world politics in recent years according to the text?',
  'Choices': 'a: The growth of nationalism

In [94]:
quiz_df = pd.DataFrame(quiz_table)
quiz_df

,MCQ,Choices,Correct
0,What is the main focus of traditional politica...,a: Global level politics | b: Local and nation...,b
1,What concept has challenged state-centric ways...,a: Globalization | b: International relations ...,a
2,What is the main focus of global politics acco...,a: Local politics | b: National politics | c: ...,c
3,What does the term 'global' mean in the contex...,a: Between nations | b: Worldwide | c: Compreh...,b
4,What significant changes have occurred in worl...,a: The growth of nationalism | b: The rise of ...,b
5,What does the term 'international' mean in the...,a: Between nations | b: Worldwide | c: Compreh...,a
6,How does global politics differ from internati...,a: Global politics focuses on regional issues ...,c


In [95]:
quiz_df.to_csv("global_politics_quiz.csv", index=False)